# dynamische interactie

Na sprint 2 van het ontwikkelen van de TBCI kwam de vraag of ook de code dynamischer gebruikt kan woorden:

In [ ]:
from pathlib import Path

from toolbox_continu_inzicht import Config

## 1. Voorbeeld met CSV - data adapter
Lees de configuratie in:

Dit leest een configuratie bestand `test_config.yaml` in. 
Hier definieren we een aantal data adapaters die lezen/schrijven mogelijk maken:

```yaml
GlobalVariables:
    rootdir: 'tests/src/base/data_sets'

DataAdapter:
    postgresql_database:
        database: 'citoolbox'
        schema: 'citoolbox_schema'
        
    my_csv_in: 
        type: csv
        file: 'test_csv_in.csv'
    my_csv_in_delim: 
        type: csv
        file: 'test_csv_in_delim.csv'
        sep: ';'
    MyCSV_out: 
        type: csv
        file: 'test_csv_out.csv'
    MyPostgresql: 
        type: postgresql_database
        table: data_test
    MyNetCDF_in:
        type: netcdf
        file: 'test_netcdf_in.nc'
    MyNetCDF_out:
        type: netcdf
        file: 'test_netcdf_out.nc'

...

```

In [ ]:
test_data_sets_path = Path.cwd() / "data_sets"
config = Config(config_path=test_data_sets_path / "test_config.yaml")
config.lees_config()

Zet de data adapter klaar en geef deze de configuratie mee 

In [ ]:
from toolbox_continu_inzicht import DataAdapter

data_adapter = DataAdapter(config=config)

Met de functie `set_global_variable` kan dit nu

In [ ]:
data_adapter.config.global_variables["calc_time"]

In [ ]:
from datetime import datetime, timedelta, timezone

dt_new_time = datetime.now() + timedelta(days=1)
new_time = datetime(
    dt_new_time.year,
    dt_new_time.month,
    dt_new_time.day,
    dt_new_time.hour,
    0,
    0,
).replace(tzinfo=timezone.utc)

data_adapter.set_global_variable("calc_time", new_time)

In [ ]:
data_adapter.config.global_variables["calc_time"]

In [ ]:
from toolbox_continu_inzicht.proof_of_concept import example_module

keer_twee = example_module.ValuesTimesTwo(data_adapter=data_adapter)

In [ ]:
keer_twee.run(input="mycsv_in", output="mycsv_out")

In [ ]:
input_df = keer_twee.df_in

In [ ]:
input_df

```yaml
GlobalVariables:
    rootdir: 'data_sets'
    moments: [-24,0,24,48] 

DataAdapter:
    default_options:        
        csv:
            sep: ","

    my_csv: 
        type: csv
        file: "test_csv_in.csv"
        sep: ";"
    my_df_python:
        type: python
    my_csv_out:   
        type: csv
        file: "test_csv_in.csv"

```

In [ ]:
config = Config(config_path=test_data_sets_path / "test_config_dynamic.yaml")
config.lees_config()

In [ ]:
data_adapter = DataAdapter(config=config)
data_adapter.set_dataframe_adapter("my_df_python", input_df)

In [ ]:
keer_twee = example_module.ValuesTimesTwo(data_adapter=data_adapter)
keer_twee.run(input="my_df_python", output="my_df_python")

In [ ]:
df_in_functie2 = keer_twee.df_out
df_in_functie2

In [ ]:
data_adapter.set_dataframe_adapter(
    "my_df_python_2", df_in_functie2, if_not_exist="create"
)

In [ ]:
data_adapter.config.data_adapters.keys()

### show how a custom data adapter works

```python
def input_test_text(input_config: dict):
    """Function to read a text file given a path"""
    # input_config contains all the information from reading the config file
    path = input_config["abs_path"]
    # we check with `get_kwargs` which options are compatible
    kwargs = get_kwargs(open, input_config)
    kwargs.pop("file")  # use the abs path instead of file
    with open(path, **kwargs) as f:
        data = f.read()
    # return the data in a dataframe, this is just an example
    return pd.DataFrame([data], columns=["text"])
```

```yaml
GlobalVariables:
    rootdir: 'data_sets'
    input_plugin_path: 'data_sets'

DataAdapter:
    example_test:
        type: test_text
        file: 'test.txt'

```

In [ ]:
config = Config(config_path=test_data_sets_path / "test_extra_da.yaml")
config.lees_config()

In [ ]:
data_adapter = DataAdapter(config=config)

In [ ]:
df_in = data_adapter.input("example_test")
df_in